In [57]:
import pandas as pd

# Leemos el CSV y evitamos que el índice se guarde como columna
df = pd.read_csv("df_200_con_huecos.csv", index_col=0)

# Si aun así quedó la columna Unnamed: 0, la borramos:
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

df


,date,rio_id,lat,lon,altura_value,precipitaciones_value
0,2023-01-01,10-10,-25.916679,-54.621134,7.80,0.0
1,2023-01-02,10-10,-25.916679,-54.621134,6.50,38.5
2,2023-01-03,10-10,-25.916679,-54.621134,7.20,22.3
3,2023-01-04,10-10,-25.916679,-54.621134,7.10,1.3
4,2023-01-05,10-10,-25.916679,-54.621134,8.10,0.0
...,...,...,...,...,...,...
190696,2025-10-01,99-99,-33.014444,-58.504167,2.06,0.0
190697,2025-10-02,99-99,-33.014444,-58.504167,2.06,0.0
190698,2025-10-03,99-99,-33.014444,-58.504167,2.00,0.0
190699,2025-10-04,99-99,-33.014444,-58.504167,1.98,0.1


## Iniciando request a Open Meteo para corregir el nro de precipitaciones.

In [47]:
def tomar_precipitaciones_bien(lat, lon):
    """Devuelve una lista 'precipitation_sum' con valores diarios desde 2023-01-01 hasta 2025-10-05.
    Si faltan días se rellenan con 0 manteniendo la posición correspondiente."""
    import requests
    import pandas as pd
    start = pd.Timestamp('2023-01-01')
    end = pd.Timestamp('2025-10-05')
    url = (
        f"https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}"
        f"&start_date={start.date()}&end_date={end.date()}&daily=precipitation_sum&timezone=auto"
    )
    try:
        resp = requests.get(url, timeout=20)
        resp.raise_for_status()
        data = resp.json()
    except Exception as e:
        print('Error al obtener datos de la API:', e)
        precipitation_sum = []
        return precipitation_sum

    daily = data.get('daily', {})
    times = daily.get('time', [])
    precs = daily.get('precipitation_sum', [])

    # Crear serie datetime -> precipitación (maneja desajustes de longitud)
    try:
        # Truncar al tamaño mínimo entre times y precs si es necesario
        n = min(len(times), len(precs))
        times_trunc = times[:n]
        precs_trunc = precs[:n]
        idx = pd.to_datetime(times_trunc, errors='coerce')
        ser = pd.Series(data=precs_trunc, index=idx)
    except Exception as e:
        print('Error al parsear arrays de daily:', e)
        ser = pd.Series(dtype=float)

    # Rango completo de fechas y reindexar rellenando con 0 donde falte
    full_index = pd.date_range(start, end, freq='D')
    ser = ser.reindex(full_index, fill_value=0)

    # Asegurar que el resultado es una lista de floats/ints con la longitud esperada
    precipitation_sum = [float(x) if pd.notna(x) else 0.0 for x in ser.tolist()]
    return precipitation_sum

In [48]:
def crear_serie_precipitaciones(arr, start='2023-01-01', col_name='precipitaciones_value'):
    """Crear DataFrame diario con columna col_name a partir de un iterable `arr`.
    - start: fecha inicial (string o datetime).
    - Si arr está vacío devuelve DataFrame vacío con la columna requerida.
    """
    import pandas as pd
    # Normalizar None/NaN y convertir a lista
    if arr is None:
        return pd.DataFrame(columns=[col_name])
    if isinstance(arr, pd.Series):
        values = arr.fillna(0).tolist()
    else:
        try:
            # evitar tratar strings como iterables de caracteres
            if isinstance(arr, (str, bytes)):
                values = [arr]
            else:
                values = [0.0 if (x is None or (isinstance(x, float) and pd.isna(x))) else x for x in arr]
        except TypeError:
            values = [arr]

    # asegurar tipos numéricos y reemplazar NaN/None por 0.0
    cleaned = []
    for v in values:
        try:
            if v is None:
                cleaned.append(0.0)
            else:
                # convertir a float si posible
                cleaned.append(float(v))
        except Exception:
            cleaned.append(0.0)

    n = len(cleaned)
    if n == 0:
        return pd.DataFrame(columns=[col_name])

    start_ts = pd.to_datetime(start)
    idx = pd.date_range(start=start_ts, periods=n, freq='D')
    df = pd.DataFrame({col_name: cleaned}, index=idx)
    # opcional: asegurar nombre del índice
    df.index.name = 'date'
    return df

In [49]:
# Celda 4 (corregida)
import pandas as pd
from tqdm import tqdm

df = df.copy()
df["precipitaciones_value"] = None  # la creamos vacía

for rio_id in tqdm(df["rio_id"].unique(), desc="Procesando ríos"):
    df_rio_mask = df["rio_id"] == rio_id
    df_rio = df[df_rio_mask].reset_index(drop=True)
    
    lat = df_rio["lat"].iloc[0]
    lon = df_rio["lon"].iloc[0]
    
    arr_precip = tomar_precipitaciones_bien(lat, lon)
    df_precip = crear_serie_precipitaciones(arr_precip, start="2023-01-01")

    # asegurar que df_precip tenga la misma longitud que df_rio
    n = min(len(df_precip), len(df_rio))
    df_precip = df_precip.iloc[:n].reset_index(drop=True)
    df_rio = df_rio.iloc[:n].reset_index(drop=True)
    
    # asignar directamente por posición (sin alinear índices)
    df.loc[df_rio_mask, "precipitaciones_value"] = df_precip["precipitaciones_value"].values[:len(df_rio_mask[df_rio_mask])]
    
print("✅ Precipitaciones integradas correctamente en el DataFrame.")


Procesando ríos: 100%|██████████| 189/189 [03:56<00:00,  1.25s/it]

✅ Precipitaciones integradas correctamente en el DataFrame.


In [50]:
df.groupby("rio_id")["precipitaciones_value"].apply(lambda x: x.isna().sum())


rio_id
10-10        0
11-11        0
12-12        0
13-13        0
14-14        0
            ..
9412-2837    0
9494-2844    0
9542-2813    0
9549-2809    0
99-99        0
Name: precipitaciones_value, Length: 189, dtype: int64

In [54]:
df

,Unnamed: 0,date,rio_id,lat,lon,altura_value,precipitaciones_value
0,0,2023-01-01,10-10,-25.916679,-54.621134,7.80,0.0
1,1,2023-01-02,10-10,-25.916679,-54.621134,6.50,38.5
2,2,2023-01-03,10-10,-25.916679,-54.621134,7.20,22.3
3,3,2023-01-04,10-10,-25.916679,-54.621134,7.10,1.3
4,4,2023-01-05,10-10,-25.916679,-54.621134,8.10,0.0
...,...,...,...,...,...,...,...
190696,190696,2025-10-01,99-99,-33.014444,-58.504167,2.06,0.0
190697,190697,2025-10-02,99-99,-33.014444,-58.504167,2.06,0.0
190698,190698,2025-10-03,99-99,-33.014444,-58.504167,2.00,0.0
190699,190699,2025-10-04,99-99,-33.014444,-58.504167,1.98,0.1


In [52]:
# df.to_csv('df_200_con_huecos.csv')